In [22]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
import numpy as np

In [3]:
iris = load_iris()

In [4]:
X = iris.data
y = iris.target

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
X.shape

(150, 4)

In [7]:
onehot = OneHotEncoder(categorical_features=[0])
y = onehot.fit_transform(y.reshape(-1,1))
y = y.toarray()

/home/gustavo/Envs/machine/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gustavo/Envs/machine/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
neuronios_input = X.shape[1]
neuronios_output = y.shape[1]
neuronios_oculta =  np.ceil((X.shape[1]+y.shape[1])/2)
neuronios_oculta = np.int32(neuronios_oculta)

In [10]:
pesos = {
    'oculta':tf.Variable(
        tf.random_normal([neuronios_input, neuronios_oculta]), dtype=tf.float32),
    'saida': tf.Variable(
        tf.random_normal([neuronios_oculta, neuronios_output]), dtype=tf.float32) 
}

bias = {
    'oculta':tf.Variable(
        tf.random_normal([neuronios_oculta])),
    'saida': tf.Variable(
        tf.random_normal([neuronios_output])) 
}

In [11]:
 # Qualquer numero de observações, mas Tem o numero de inputs definido antes
xph = tf.placeholder(dtype=tf.float32, shape=[None, neuronios_input])

# qualquer numero de observações, mas tem 3 saídas por causa do OnehotEncoder
# Essa rede tem 3 saídas. São equivalentes as probabilidades de cada classe
yph = tf.placeholder(dtype=tf.float32, shape=[None, neuronios_output])

In [12]:
def multilayer_perceptron(inputs, weights, bias):
    # multiplicacao dos pesos pelas features
    valores_camada_oculta = tf.matmul(inputs, weights['oculta'])
    
    #adição do bias
    valores_camada_oculta = tf.add(valores_camada_oculta, bias['oculta'])
    
    #Função de ativacao
    valores_camada_oculta = tf.nn.relu(valores_camada_oculta)
    
    #mesmo processo anterior.
    valores_camada_saida = tf.matmul(valores_camada_oculta, weights['saida'])
    valores_camada_saida = tf.add(valores_camada_saida, bias['saida'])
#     valores_camada_saida = tf.nn.softmax(valores_camada_saida)
    
    return valores_camada_saida

In [13]:
modelo = multilayer_perceptron(xph, pesos, bias)

In [14]:
# Definição do erro

erro  = tf.nn.softmax_cross_entropy_with_logits_v2(logits=modelo, labels=yph)
erro = tf.reduce_mean(erro)
otimizador = tf.train.AdamOptimizer().minimize(erro)


In [15]:
BATCH_SIZE = 8
TOTAL_ITERACOES = int(len(X_train)/BATCH_SIZE)
EPOCAS = 3000


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoca in range(EPOCAS):
        erro_medio = 0.0
        
        for i in range(TOTAL_ITERACOES):
            x = X_train[BATCH_SIZE*i:BATCH_SIZE*(i+1)]
            y = y_train[BATCH_SIZE*i:BATCH_SIZE*(i+1)]
            _, custo = sess.run([otimizador, erro], feed_dict={xph:x, yph:y})
            erro_medio += custo/TOTAL_ITERACOES
        if epoca % 500 == 0:
            print("Epoca: ", epoca, " erro: ", erro_medio)
    pesos_finais, bias_final = sess.run([pesos, bias])

Epoca:  0  erro:  10.636483473437172
Epoca:  500  erro:  0.0664040577331824
Epoca:  1000  erro:  0.03971855766576482
Epoca:  1500  erro:  0.03401894936216975
Epoca:  2000  erro:  0.032239870507851265
Epoca:  2500  erro:  0.03139297587346894


In [16]:
print(pesos_finais, bias_final)

{'saida': array([[  0.21316461,  -1.0615246 ,  -0.5832064 ],
       [  4.2961926 ,   1.3000625 , -11.312548  ],
       [ -9.123332  ,   0.06577493,   0.9042644 ],
       [  0.23410052,   2.1235445 ,  -5.2911305 ]], dtype=float32), 'oculta': array([[ 0.52425224, -1.9160359 ,  0.9959063 ,  0.01090503],
       [-0.36845785,  1.6150159 , -1.2740028 ,  0.06687524],
       [-1.341526  , -1.1911432 ,  2.0890374 , -1.1061895 ],
       [ 0.8445673 , -1.9278212 ,  2.908283  , -1.2803642 ]],
      dtype=float32)} {'saida': array([0.6012882 , 0.28831527, 0.7959783 ], dtype=float32), 'oculta': array([-0.7667937,  2.7576663,  2.293447 ,  2.217557 ], dtype=float32)}


In [17]:
# previsão

previsao_teste = multilayer_perceptron(xph, pesos_finais, bias_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev = sess.run(previsao_teste, feed_dict={xph: X_test, yph:y_test})
    #Aplicação da função de probabilidde
    prev = sess.run(tf.nn.softmax(prev))
    
    #Pegando qual coluna tem o valor mais alto, pois, o indice dessa coluna
    # indica qual é a categoria da previsão. (Lembre-se do OneHotEncoder definido antes)
    prev = sess.run(tf.arg_max(prev, 1)) 

Instructions for updating:
Use `argmax` instead


In [24]:
print(prev)
print(np.argmax(y_test, 1))


print(classification_report(np.argmax(y_test, 1), prev))

[1 1 2 0 0 2 2 1 0 2 1 0 1 2 2 0 1 0 0 1 1 2 1 0 2 1 2 0 2 2 2 0 2 2 2 1 2
 1]
[1 1 2 0 0 2 2 1 0 2 1 0 2 2 2 0 1 0 0 1 1 2 1 0 2 1 2 0 2 2 1 0 2 2 2 1 2
 1]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.92      0.92      0.92        12
           2       0.94      0.94      0.94        16

   micro avg       0.95      0.95      0.95        38
   macro avg       0.95      0.95      0.95        38
weighted avg       0.95      0.95      0.95        38

